![lop](../../images/logo_diive1_128px.png)

<span style='font-size:32px; display:block;'>
<b>
    Resampling: Diel cycle
</b>
</span>

---
**Notebook version**: `1` (14 May 2024)  
**Author**: Lukas Hörtnagl (holukas@ethz.ch)  

</br>

# **Description**

- `DielCycle` is a class for plot diel cycles of time series data.
- Uses the time info from the timestamp (datetime) index to aggregate. Aggregation is done on unique times found in the index.
- For example, if the time part of the datetime index is hourly (e.g. 0:00, 01:00, 02:00 etc.), then hourly averages are calculated.

</br>

# **Imports**

In [1]:
import importlib.metadata
import warnings
from datetime import datetime

import diive.configs.exampledata as ed
from diive.core.times.resampling import diel_cycle

warnings.filterwarnings('ignore')
version_diive = importlib.metadata.version("diive")
print(f"diive version: v{version_diive}")

diive version: v0.79.0


</br>

# **Docstring**

In [2]:
help(diel_cycle)

Help on function diel_cycle in module diive.core.times.resampling:

diel_cycle(series: pandas.core.series.Series, mincounts: int = 1, mean: bool = True, std: bool = True, median: bool = False, each_month: bool = False) -> pandas.core.frame.DataFrame
    Calculate diel cycles grouped by time



</br>

# **Load example data**

- Load CO2 and latent evaporation as example data

In [3]:
df = ed.load_exampledata_parquet()
co2 = df['NEE_CUT_REF_f'].copy()
co2

Loaded .parquet file F:\Sync\luhk_work\20 - CODING\21 - DIIVE\diive\diive\configs\exampledata\exampledata_PARQUET_CH-DAV_FP2022.5_2013-2022_ID20230206154316_30MIN.parquet (0.072 seconds). Detected time resolution of <30 * Minutes> / 30min 


TIMESTAMP_MIDDLE
2013-01-01 00:15:00    1.595
2013-01-01 00:45:00    1.595
2013-01-01 01:15:00    1.595
2013-01-01 01:45:00    0.538
2013-01-01 02:15:00    1.765
                       ...  
2022-12-31 21:45:00    1.224
2022-12-31 22:15:00    3.518
2022-12-31 22:45:00    2.424
2022-12-31 23:15:00    1.017
2022-12-31 23:45:00    2.281
Freq: 30min, Name: NEE_CUT_REF_f, Length: 175296, dtype: float64

</br>

# **Calculate diel cycle**

## Monthly diel cycles
- Monthly diel cycles are stored using a MultiIndex where the month is stored as its integer number (e.g. `1` for January), followed by the aggregates grouped by time.

In [4]:
aggs = diel_cycle(
    series=co2,
    mincounts=1,
    mean=True,
    std=True,
    median=True,
    each_month=True
)

In [5]:
aggs

count      mean       std  median   mean+sd   mean-sd  median+sd  median-sd
TIMESTAMP_MIDDLE                                                                                      
1                00:15:00    310  1.028971  1.217350  0.9235  2.246321 -0.188379   2.140850  -0.293850
                 00:45:00    310  1.039523  0.930382  0.9235  1.969905  0.109140   1.853882  -0.006882
                 01:15:00    310  1.002735  1.053311  0.9085  2.056047 -0.050576   1.961811  -0.144811
                 01:45:00    310  0.976465  1.059341  0.8505  2.035806 -0.082877   1.909841  -0.208841
                 02:15:00    310  0.979710  1.053724  0.9095  2.033434 -0.074014   1.963224  -0.144224
...                          ...       ...       ...     ...       ...       ...        ...        ...
12               21:45:00    310  1.113068  1.043182  1.2145  2.156250  0.069886   2.257682   0.171318
                 22:15:00    310  1.214081  1.414525  1.1800  2.628606 -0.200444   2.594525  -0.234525
                 22:45:00    310  1.206858  1.049283  1.2010  2.256141  0.157575   2.250283   0.151717
                 23:15:00    310  1.158884  1.197403  1.2070  2.356287 -0.038519   2.404403   0.009597
                 23:45:00    310  1.181277  1.026592  1.2130  2.207869  0.154685   2.239592   0.186408

[576 rows x 8 columns]

Show data for January:

In [6]:
aggs.loc[1].head(5)

,count,mean,std,median,mean+sd,mean-sd,median+sd,median-sd
00:15:00,310,1.028971,1.217350,0.9235,2.246321,-0.188379,2.140850,-0.293850
00:45:00,310,1.039523,0.930382,0.9235,1.969905,0.109140,1.853882,-0.006882
01:15:00,310,1.002735,1.053311,0.9085,2.056047,-0.050576,1.961811,-0.144811
01:45:00,310,0.976465,1.059341,0.8505,2.035806,-0.082877,1.909841,-0.208841
02:15:00,310,0.979710,1.053724,0.9095,2.033434,-0.074014,1.963224,-0.144224


Show data for June:

In [7]:
aggs.loc[6].head(5)

,count,mean,std,median,mean+sd,mean-sd,median+sd,median-sd
00:15:00,300,4.665877,2.219011,4.4565,6.884888,2.446866,6.675511,2.237489
00:45:00,300,4.529247,2.131744,4.3415,6.660991,2.397502,6.473244,2.209756
01:15:00,300,4.587670,1.987093,4.3150,6.574763,2.600577,6.302093,2.327907
01:45:00,300,4.554280,2.077765,4.3475,6.632045,2.476515,6.425265,2.269735
02:15:00,300,4.537543,2.009197,4.3545,6.546740,2.528346,6.363697,2.345303


</br>

## Diel cylce across all data

In [8]:
aggs = diel_cycle(
    series=co2,
    mincounts=1,
    mean=True,
    std=True,
    median=True,
    each_month=False
)

In [9]:
aggs

count      mean       std  median   mean+sd    mean-sd  median+sd  median-sd
TIMESTAMP_MIDDLE                                                                                       
ALL_MONTHS       00:15:00   3652  2.628153  2.292058  2.0710  4.920210   0.336095   4.363058  -0.221058
                 00:45:00   3652  2.660316  2.251208  2.1045  4.911524   0.409109   4.355708  -0.146708
                 01:15:00   3652  2.636666  2.313408  2.1165  4.950074   0.323259   4.429908  -0.196908
                 01:45:00   3652  2.609721  2.221190  2.0755  4.830911   0.388532   4.296690  -0.145690
                 02:15:00   3652  2.600780  2.252368  2.0600  4.853148   0.348412   4.312368  -0.192368
                 02:45:00   3652  2.628637  2.295040  2.0960  4.923677   0.333597   4.391040  -0.199040
                 03:15:00   3652  2.572976  2.241159  2.0515  4.814135   0.331816   4.292659  -0.189659
                 03:45:00   3652  2.542711  2.206123  2.0320  4.748834   0.336589   4.238123  -0.174123
                 04:15:00   3652  2.518468  2.145243  2.0070  4.663710   0.373225   4.152243  -0.138243
                 04:45:00   3652  2.490877  2.216896  1.9990  4.707773   0.273981   4.215896  -0.217896
                 05:15:00   3652  2.162079  2.357829  1.7720  4.519909  -0.195750   4.129829  -0.585829
                 05:45:00   3652  1.540008  2.825489  1.4505  4.365497  -1.285480   4.275989  -1.374989
                 06:15:00   3652  0.812312  3.235995  1.1370  4.048307  -2.423683   4.372995  -2.098995
                 06:45:00   3652  0.184065  3.329848  0.8400  3.513913  -3.145783   4.169848  -2.489848
                 07:15:00   3652 -0.429212  3.634704  0.5195  3.205491  -4.063916   4.154204  -3.115204
                 07:45:00   3652 -1.615019  4.421238 -0.1375  2.806220  -6.036257   4.283738  -4.558738
                 08:15:00   3652 -3.062653  5.305186 -1.4945  2.242533  -8.367839   3.810686  -6.799686
                 08:45:00   3652 -4.042698  5.514220 -3.3345  1.471523  -9.556918   2.179720  -8.848720
                 09:15:00   3652 -4.577710  5.651760 -4.1465  1.074050 -10.229470   1.505260  -9.798260
                 09:45:00   3652 -4.992462  5.872298 -4.7230  0.879836 -10.864760   1.149298 -10.595298
                 10:15:00   3652 -5.340606  5.885783 -5.0780  0.545178 -11.226389   0.807783 -10.963783
                 10:45:00   3652 -5.474241  5.918189 -5.3830  0.443948 -11.392430   0.535189 -11.301189
                 11:15:00   3652 -5.511982  5.981464 -5.4905  0.469482 -11.493446   0.490964 -11.471964
                 11:45:00   3652 -5.488146  5.999366 -5.3755  0.511220 -11.487513   0.623866 -11.374866
                 12:15:00   3652 -5.415469  5.828823 -5.1750  0.413355 -11.244292   0.653823 -11.003823
                 12:45:00   3652 -5.213165  5.753296 -5.1625  0.540131 -10.966461   0.590796 -10.915796
                 13:15:00   3652 -5.125854  5.612403 -5.0400  0.486549 -10.738256   0.572403 -10.652403
                 13:45:00   3652 -4.871877  5.358231 -4.5985  0.486354 -10.230108   0.759731  -9.956731
                 14:15:00   3652 -4.576961  5.257046 -4.2625  0.680085  -9.834007   0.994546  -9.519546
                 14:45:00   3652 -4.086386  5.077501 -3.6555  0.991115  -9.163886   1.422001  -8.733001
                 15:15:00   3652 -3.683662  4.878181 -3.1800  1.194519  -8.561844   1.698181  -8.058181
                 15:45:00   3652 -3.116265  4.693900 -2.6060  1.577635  -7.810164   2.087900  -7.299900
                 16:15:00   3652 -2.441634  4.421946 -1.8065  1.980312  -6.863579   2.615446  -6.228446
                 16:45:00   3652 -1.674700  4.368864 -0.8170  2.694165  -6.043564   3.551864  -5.185864
                 17:15:00   3652 -0.935853  3.872267 -0.1090  2.936414  -4.808120   3.763267  -3.981267
                 17:45:00   3652 -0.102077  3.513286  0.5975  3.411209  -3.615363   4.110786  -2.915786
                 18:15:00   3652  0.609048  3.496362  1.0170  4.105410  -2.

</br>

# End of notebook

In [10]:
dt_string = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Finished {dt_string}")

Finished 2024-08-22 15:44:20
